# Ejercicio 3: Sistema de Metro - BFS vs IDS

Implementación y comparación de algoritmos de búsqueda para encontrar la ruta más corta entre estaciones A y J.


In [1]:
import time
import tracemalloc
from collections import deque


In [2]:
# Todas las clases y funciones necesarias
class Problem: 
    def __init__(self, initial, goal, graph):
        self.initial = initial
        self.goal = goal
        self.graph = graph
    
    def actions(self, state):
        return self.graph.get(state, [])
    
    def goal_test(self, state):
        return state == self.goal

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.depth = 0 if parent is None else parent.depth + 1
        
    def path(self):
        node, path = self, []
        while node is not None:
            path.append(node.state)
            node = node.parent
        return list(reversed(path))
    
    def __repr__(self):
        return f"Node({self.state!r}, depth={self.depth})"


In [3]:
# Implementación BFS
def bfs(problem):
    root = Node(problem.initial)
    if problem.goal_test(root.state):
        return root

    frontier = deque([root])
    visited = {root.state}

    while frontier:
        node = frontier.popleft()
        
        for neighbor in problem.actions(node.state):
            if neighbor in visited:
                continue
            child = Node(neighbor, parent=node)

            if problem.goal_test(child.state):
                return child

            visited.add(neighbor)
            frontier.append(child)

    return None


In [4]:
# Implementación IDS
def depth_limited_search(problem, limit):
    def recursive_dls(node, limit, path_set):
        if problem.goal_test(node.state):
            return node
        
        if limit == 0:
            return None
        
        for neighbor in problem.actions(node.state):
            if neighbor in path_set:
                continue
                
            child = Node(neighbor, parent=node)
            result = recursive_dls(child, limit - 1, path_set | {neighbor})
            
            if result is not None:
                return result
        
        return None
    
    root = Node(problem.initial)
    return recursive_dls(root, limit, {problem.initial})

def iterative_deepening_search(problem, max_depth=50):
    for depth in range(max_depth + 1):
        result = depth_limited_search(problem, depth)
        if result is not None:
            return result
    return None


In [5]:
# Grafo del metro
graph = {
    'A': ['B','C'],
    'B': ['A','D','E'],   
    'C': ['A','F'], 
    'D': ['B','G'], 
    'E': ['B','H','I'], 
    'F': ['C','J'], 
    'G': ['D'],
    'H': ['E'], 
    'I': ['E','J'], 
    'J': ['F','I'],     
}


In [6]:
# Función para medir métricas
def run_with_metrics(search_fn, problem, *args, **kwargs):
    """Ejecuta una función de búsqueda y mide tiempo y memoria"""
    tracemalloc.start()
    start_time = time.perf_counter()
    
    result = search_fn(problem, *args, **kwargs)
    
    end_time = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    elapsed_time = end_time - start_time
    return result, elapsed_time, peak


In [7]:
# Ejecutar ambos algoritmos
problem = Problem("A", "J", graph)

print("COMPARACIÓN DE ALGORITMOS DE BÚSQUEDA")
print("=" * 50)
print(f"Origen: {problem.initial}")
print(f"Destino: {problem.goal}")
print("=" * 50)

# Ejecutar BFS
print("\nBREADTH-FIRST SEARCH (BFS)")
print("-" * 30)
solution_bfs, time_bfs, memory_bfs = run_with_metrics(bfs, problem)

if solution_bfs:
    print("Solución encontrada")
    print(f"Ruta: {' → '.join(solution_bfs.path())}")
    print(f"Número de acciones: {solution_bfs.depth}")
    print(f"Tiempo de ejecución: {time_bfs:.6f} segundos")
    print(f"Memoria pico: {memory_bfs:,} bytes ({memory_bfs/1024:.2f} KB)")
else:
    print("No se encontró solución")

# Ejecutar IDS
print("\nITERATIVE DEEPENING SEARCH (IDS)")
print("-" * 30)
solution_ids, time_ids, memory_ids = run_with_metrics(iterative_deepening_search, problem)

if solution_ids:
    print("Solución encontrada")
    print(f"Ruta: {' → '.join(solution_ids.path())}")
    print(f"Número de acciones: {solution_ids.depth}")
    print(f"Tiempo de ejecución: {time_ids:.6f} segundos")
    print(f"Memoria pico: {memory_ids:,} bytes ({memory_ids/1024:.2f} KB)")
else:
    print("No se encontró solución")


COMPARACIÓN DE ALGORITMOS DE BÚSQUEDA
Origen: A
Destino: J

BREADTH-FIRST SEARCH (BFS)
------------------------------
Solución encontrada
Ruta: A → C → F → J
Número de acciones: 3
Tiempo de ejecución: 0.000791 segundos
Memoria pico: 4,752 bytes (4.64 KB)

ITERATIVE DEEPENING SEARCH (IDS)
------------------------------
Solución encontrada
Ruta: A → C → F → J
Número de acciones: 3
Tiempo de ejecución: 0.000302 segundos
Memoria pico: 2,704 bytes (2.64 KB)


In [8]:
# Datos comparativos
if solution_bfs and solution_ids:
    print("\nDATOS COMPARATIVOS")
    print("=" * 50)
    
    print("Número de acciones:")
    print(f"   BFS: {solution_bfs.depth}")
    print(f"   IDS: {solution_ids.depth}")
    
    print("\nRutas encontradas:")
    print(f"   BFS: {' → '.join(solution_bfs.path())}")
    print(f"   IDS: {' → '.join(solution_ids.path())}")
    
    print("\nTiempo de ejecución:")
    print(f"   BFS: {time_bfs:.6f} segundos")
    print(f"   IDS: {time_ids:.6f} segundos")
    
    print("\nUso de memoria:")
    print(f"   BFS: {memory_bfs:,} bytes ({memory_bfs/1024:.2f} KB)")
    print(f"   IDS: {memory_ids:,} bytes ({memory_ids/1024:.2f} KB)")



DATOS COMPARATIVOS
Número de acciones:
   BFS: 3
   IDS: 3

Rutas encontradas:
   BFS: A → C → F → J
   IDS: A → C → F → J

Tiempo de ejecución:
   BFS: 0.000791 segundos
   IDS: 0.000302 segundos

Uso de memoria:
   BFS: 4,752 bytes (4.64 KB)
   IDS: 2,704 bytes (2.64 KB)
